## Customer Analysis Case Study

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd

### Loading Data

In [2]:
# Load datasets individually
file1 = pd.read_csv('Data/file1.csv')
file2 = pd.read_csv('Data/file2.csv')
file3 = pd.read_csv('Data/file3.csv')

In [3]:
# Check if the datasets have the same column names
file1_colnames = sorted(list(file1.columns))
file2_colnames = sorted(list(file2.columns))
file3_colnames = sorted(list(file3.columns))
print(file1_colnames, file2_colnames, file3_colnames, sep='\n')

['Customer', 'Customer Lifetime Value', 'Education', 'GENDER', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'ST', 'Total Claim Amount', 'Vehicle Class']
['Customer', 'Customer Lifetime Value', 'Education', 'GENDER', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'ST', 'Total Claim Amount', 'Vehicle Class']
['Customer', 'Customer Lifetime Value', 'Education', 'Gender', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'State', 'Total Claim Amount', 'Vehicle Class']


In [4]:
# There are several colnames in file3 are not align with which of the others

file3.rename(columns={'Gender':'GENDER', 'State':'ST'}, inplace=True )

file1_colnames = sorted(list(file1.columns))
file2_colnames = sorted(list(file2.columns))
file3_colnames = sorted(list(file3.columns))
print(file1_colnames, file2_colnames, file3_colnames, sep='\n')

['Customer', 'Customer Lifetime Value', 'Education', 'GENDER', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'ST', 'Total Claim Amount', 'Vehicle Class']
['Customer', 'Customer Lifetime Value', 'Education', 'GENDER', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'ST', 'Total Claim Amount', 'Vehicle Class']
['Customer', 'Customer Lifetime Value', 'Education', 'GENDER', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'ST', 'Total Claim Amount', 'Vehicle Class']


### Concatenating Data

In [5]:
ca_df = pd.concat([file1,file2,file3], axis=0)
ca_df

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
7065,LA72316,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


### Standardizing Column Names

In [6]:
def standardize_column_names(ca_df):
    ca_df.columns = ["_".join(i.lower().split()) for i in ca_df.columns]
    return ca_df

In [7]:
ca_df = standardize_column_names(ca_df)

In [8]:
ca_df

,customer,st,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
7065,LA72316,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


### Renaming Column

In [9]:
def rename_columns(ca_df):
    ca_df.rename(columns={'st':'state'}, inplace=True )
    return ca_df

In [10]:
rename_columns(ca_df)
ca_df

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
7065,LA72316,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


### Removing Duplicates

In [11]:
ca_df.drop_duplicates(inplace=True)
ca_df

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
7065,LA72316,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


### Deleting Column

In [12]:
# Delete the column `customer` as it is only a unique identifier for each row of data
def drop_columns(ca_df) :
    ca_df.drop(columns=['customer'], inplace=True)
    return ca_df

In [13]:
drop_columns(ca_df)

,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...
7065,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


### Rearanging Columns

In [14]:
list(ca_df.columns)

['state',
 'gender',
 'education',
 'customer_lifetime_value',
 'income',
 'monthly_premium_auto',
 'number_of_open_complaints',
 'policy_type',
 'vehicle_class',
 'total_claim_amount']

In [15]:
# Categorical data before numeric_data
ca_df = ca_df[['state',
               'gender',
               'education',
               'policy_type', 
               'vehicle_class',
               'number_of_open_complaints', 
               'customer_lifetime_value',
               'income',
               'monthly_premium_auto',
               'total_claim_amount']]

ca_df.head()

,state,gender,education,policy_type,vehicle_class,number_of_open_complaints,customer_lifetime_value,income,monthly_premium_auto,total_claim_amount
0,Washington,NaN,Master,Personal Auto,Four-Door Car,1/0/00,NaN,0.0,1000.0,2.704934
1,Arizona,F,Bachelor,Personal Auto,Four-Door Car,1/0/00,697953.59%,0.0,94.0,1131.464935
2,Nevada,F,Bachelor,Personal Auto,Two-Door Car,1/0/00,1288743.17%,48767.0,108.0,566.472247
3,California,M,Bachelor,Corporate Auto,SUV,1/0/00,764586.18%,0.0,106.0,529.881344
4,Washington,M,High School or Below,Personal Auto,Four-Door Car,1/0/00,536307.65%,36357.0,68.0,17.269323


### Correcting Data Types

#### Correcting Numerical Data

In [16]:
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9135 entries, 0 to 7069
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   state                      9134 non-null   object 
 1   gender                     9012 non-null   object 
 2   education                  9134 non-null   object 
 3   policy_type                9134 non-null   object 
 4   vehicle_class              9134 non-null   object 
 5   number_of_open_complaints  9134 non-null   object 
 6   customer_lifetime_value    9127 non-null   object 
 7   income                     9134 non-null   float64
 8   monthly_premium_auto       9134 non-null   float64
 9   total_claim_amount         9134 non-null   float64
dtypes: float64(3), object(7)
memory usage: 785.0+ KB


In [17]:
# Remove the percentage from the customer lifetime value and truncate it to an integer value
ca_df["customer_lifetime_value"] = pd.to_numeric(ca_df["customer_lifetime_value"].str.rstrip("%"), errors='coerce').round()

ca_df

,state,gender,education,policy_type,vehicle_class,number_of_open_complaints,customer_lifetime_value,income,monthly_premium_auto,total_claim_amount
0,Washington,NaN,Master,Personal Auto,Four-Door Car,1/0/00,NaN,0.0,1000.0,2.704934
1,Arizona,F,Bachelor,Personal Auto,Four-Door Car,1/0/00,697954.0,0.0,94.0,1131.464935
2,Nevada,F,Bachelor,Personal Auto,Two-Door Car,1/0/00,1288743.0,48767.0,108.0,566.472247
3,California,M,Bachelor,Corporate Auto,SUV,1/0/00,764586.0,0.0,106.0,529.881344
4,Washington,M,High School or Below,Personal Auto,Four-Door Car,1/0/00,536308.0,36357.0,68.0,17.269323
...,...,...,...,...,...,...,...,...,...,...
7065,California,M,Bachelor,Personal Auto,Four-Door Car,0,NaN,71941.0,73.0,198.234764
7066,California,F,College,Corporate Auto,Four-Door Car,0,NaN,21604.0,79.0,379.200000
7067,California,M,Bachelor,Corporate Auto,Four-Door Car,3,NaN,0.0,85.0,790.784983
7068,California,M,College,Personal Auto,Four-Door Car,0,NaN,21941.0,96.0,691.200000


In [18]:
# Clean the column `number_of_open_complaints` and extract the middle number which is changing between records

ca_df.loc[ca_df['number_of_open_complaints'].notna() & \
          ca_df['number_of_open_complaints'].str.contains("/"), "number_of_open_complaints"] = \
ca_df.loc[ca_df['number_of_open_complaints'].notna() & \
          ca_df['number_of_open_complaints'].str.contains("/"), "number_of_open_complaints"].apply(lambda x: x[2])

ca_df["number_of_open_complaints"] = pd.to_numeric(ca_df["number_of_open_complaints"], errors='coerce')

ca_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9135 entries, 0 to 7069
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   state                      9134 non-null   object 
 1   gender                     9012 non-null   object 
 2   education                  9134 non-null   object 
 3   policy_type                9134 non-null   object 
 4   vehicle_class              9134 non-null   object 
 5   number_of_open_complaints  9134 non-null   float64
 6   customer_lifetime_value    2057 non-null   float64
 7   income                     9134 non-null   float64
 8   monthly_premium_auto       9134 non-null   float64
 9   total_claim_amount         9134 non-null   float64
dtypes: float64(5), object(5)
memory usage: 785.0+ KB


#### Correcting Categorical Data¶

In [19]:
# Check `state` column
ca_df.state.unique()

array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon', 'Cali',
       'AZ', 'WA', nan], dtype=object)

In [20]:
def clean_state(x):
    if x in ['Cali']:
        return 'California'
    elif x in ['AZ']:
        return 'Arizona'
    elif x in ['WA']:
        return 'Washington'
    else:
        return x
    
ca_df['state'] = list(map(clean_state, ca_df['state'])) 
ca_df.state.unique()

array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon', nan],
      dtype=object)

In [21]:
# Check `gender` column
ca_df.gender.unique()

array([nan, 'F', 'M', 'Femal', 'Male', 'female'], dtype=object)

In [22]:
def clean_gender(x):
    if x in ['M']:
        return 'Male'
    elif x in ['F', 'Femal',"female"]:
        return 'Female'
    else:
        return x

ca_df['gender'] = list(map(clean_gender, ca_df['gender'])) 
ca_df.gender.unique()

array([nan, 'Female', 'Male'], dtype=object)

In [23]:
# Check `education` column
ca_df.education.unique()

array(['Master', 'Bachelor', 'High School or Below', 'College',
       'Bachelors', 'Doctor', nan], dtype=object)

In [24]:
# Check `policy_type` column
ca_df.policy_type.unique()

array(['Personal Auto', 'Corporate Auto', 'Special Auto', nan],
      dtype=object)

In [25]:
# Check `vehicle_class` column
ca_df.vehicle_class.unique()

array(['Four-Door Car', 'Two-Door Car', 'SUV', 'Luxury SUV', 'Sports Car',
       'Luxury Car', nan], dtype=object)

In [26]:
ca_df

,state,gender,education,policy_type,vehicle_class,number_of_open_complaints,customer_lifetime_value,income,monthly_premium_auto,total_claim_amount
0,Washington,NaN,Master,Personal Auto,Four-Door Car,0.0,NaN,0.0,1000.0,2.704934
1,Arizona,Female,Bachelor,Personal Auto,Four-Door Car,0.0,697954.0,0.0,94.0,1131.464935
2,Nevada,Female,Bachelor,Personal Auto,Two-Door Car,0.0,1288743.0,48767.0,108.0,566.472247
3,California,Male,Bachelor,Corporate Auto,SUV,0.0,764586.0,0.0,106.0,529.881344
4,Washington,Male,High School or Below,Personal Auto,Four-Door Car,0.0,536308.0,36357.0,68.0,17.269323
...,...,...,...,...,...,...,...,...,...,...
7065,California,Male,Bachelor,Personal Auto,Four-Door Car,0.0,NaN,71941.0,73.0,198.234764
7066,California,Female,College,Corporate Auto,Four-Door Car,0.0,NaN,21604.0,79.0,379.200000
7067,California,Male,Bachelor,Corporate Auto,Four-Door Car,3.0,NaN,0.0,85.0,790.784983
7068,California,Male,College,Personal Auto,Four-Door Car,0.0,NaN,21941.0,96.0,691.200000


In [27]:
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9135 entries, 0 to 7069
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   state                      9134 non-null   object 
 1   gender                     9012 non-null   object 
 2   education                  9134 non-null   object 
 3   policy_type                9134 non-null   object 
 4   vehicle_class              9134 non-null   object 
 5   number_of_open_complaints  9134 non-null   float64
 6   customer_lifetime_value    2057 non-null   float64
 7   income                     9134 non-null   float64
 8   monthly_premium_auto       9134 non-null   float64
 9   total_claim_amount         9134 non-null   float64
dtypes: float64(5), object(5)
memory usage: 785.0+ KB
